# MITRE ATT&CK Mapper for Windows Security Events - Demo Notebook

This notebook demonstrates how to use the `mitre_mapper_windows.py` script to automatically map Windows Security Event Log entries to MITRE ATT&CK framework tactics and techniques.

## Overview

The MITRE ATT&CK Mapper is a security automation tool that:
- Reads Windows Security event logs from the local system or CSV files
- Automatically maps Event IDs to MITRE ATT&CK techniques using dynamic rules from the Sigma repository
- Enriches events with threat intelligence context
- Generates comprehensive security reports

## Benefits of Using This Tool

### 1. **Automated Threat Intelligence Context**
   - Automatically associates Windows events with known attack techniques
   - Provides immediate context about what an event might indicate
   - No need to manually look up Event IDs in documentation

### 2. **Prioritization Through Severity Mapping**
   - Events are automatically assigned severity levels (Critical, High, Medium, Low, Informational)
   - Helps security analysts focus on the most important events first
   - Enables automated triage workflows

### 3. **Standardized Threat Framework Alignment**
   - Maps events to the industry-standard MITRE ATT&CK framework
   - Enables consistent threat reporting and communication
   - Facilitates threat hunting and detection engineering

### 4. **Dynamic Rule Updates**
   - Automatically fetches the latest detection rules from the Sigma repository
   - Stays current with emerging threats and detection techniques
   - Caches rules locally for offline operation

### 5. **Comprehensive Event Enrichment**
   - Adds tactic information (Initial Access, Execution, Persistence, etc.)
   - Includes technique descriptions and references
   - Preserves original event data for full context

### 6. **Scalable Analysis**
   - Processes large volumes of events efficiently
   - Supports both real-time (Event Log) and batch (CSV) processing
   - Generates structured reports for further analysis

Let's walk through a practical example!

## Step 1: Import Required Modules

First, we need to import the necessary components from the `mitre_mapper_windows.py` script. We'll use the key functions and classes to process events programmatically.

In [ ]:
import sys
from pathlib import Path
import pandas as pd

# Import components from the mitre_mapper_windows script
from mitre_mapper_windows import (
    DynamicRuleEngine,
    parse_windows_csv,
    apply_rules,
    write_csv_report
)

## Step 2: Initialize the Dynamic Rule Engine

The `DynamicRuleEngine` is the core component that:
- Fetches Sigma detection rules from GitHub (or loads from cache)
- Maps Event IDs to MITRE ATT&CK techniques
- Retrieves detailed technique information from the MITRE ATT&CK framework

**Note**: On first run, this will download rules and ATT&CK data. Subsequent runs use cached data for faster startup.

In [ ]:
# Initialize the rule engine
# This will load cached rules if available, or fetch from GitHub on first run
print("Initializing Dynamic Rule Engine...")
rule_engine = DynamicRuleEngine(refresh_rules=False)  # Set to True to force refresh
print("Rule engine initialized successfully!")

## Step 3: Load and Examine Sample Security Events

Let's load the sample security events CSV file to see what we're working with. This file contains various Windows Security events that we'll enrich with MITRE ATT&CK context.

In [ ]:
# Load the sample CSV file
csv_path = "TEST_SECURITY_EVENTS.csv"

# Read the original CSV to see what we're starting with
print(f"Loading events from {csv_path}...")
original_events = pd.read_csv(csv_path)
print(f"\nLoaded {len(original_events)} events")
print("\nOriginal Event IDs in the file:")
print(original_events['Event ID'].value_counts().sort_index())
print("\nFirst few events:")
original_events.head()

## Step 4: Process Events with MITRE ATT&CK Mapping

Now we'll process each event through the rule engine to enrich them with MITRE ATT&CK information. This is where the magic happens - each Event ID is matched against known attack techniques.

In [ ]:
# Parse events from CSV
parsed_events = list(parse_windows_csv(csv_path))
print(f"Parsed {len(parsed_events)} events from CSV")

# Apply MITRE ATT&CK rules to each event
print("\nEnriching events with MITRE ATT&CK mappings...")
enriched_events = []
for event in parsed_events:
    enriched = apply_rules(event, rule_engine)
    enriched_events.append(enriched)

print(f"Enriched {len(enriched_events)} events")

## Step 5: Analyze the Enriched Results

Let's examine what MITRE ATT&CK information was added to our events. This shows how the tool transforms raw Windows events into actionable threat intelligence.

In [ ]:
# Convert to DataFrame for easier analysis
enriched_df = pd.DataFrame(enriched_events)

# Show events that were successfully mapped to MITRE ATT&CK
mapped_events = enriched_df[enriched_df['technique_id'] != '']
print(f"Events mapped to MITRE ATT&CK: {len(mapped_events)} out of {len(enriched_df)}")
print("\n" + "="*80)
print("ENRICHED EVENTS WITH MITRE ATT&CK MAPPINGS")
print("="*80)

# Display key enrichment fields
display_columns = ['timestamp', 'host', 'event_id', 'technique_id', 'technique_name', 
                  'tactic', 'severity', 'description']
mapped_events[display_columns].head(10)

## Step 6: Threat Intelligence Summary

Let's create a summary of the threats detected, organized by MITRE ATT&CK tactics and techniques. This demonstrates how the tool helps prioritize and understand security events.

In [ ]:
# Summary by MITRE ATT&CK Tactic
print("MITRE ATT&CK TACTIC SUMMARY")
print("="*80)
tactic_summary = mapped_events.groupby('tactic').agg({
    'event_id': 'count',
    'technique_id': lambda x: ', '.join(x.unique()),
    'severity': lambda x: ', '.join(x.unique())
}).rename(columns={'event_id': 'event_count'})
tactic_summary = tactic_summary.sort_values('event_count', ascending=False)
print(tactic_summary)

print("\n" + "="*80)
print("MITRE ATT&CK TECHNIQUE SUMMARY")
print("="*80)
technique_summary = mapped_events.groupby(['technique_id', 'technique_name', 'tactic', 'severity']).size().reset_index(name='count')
technique_summary = technique_summary.sort_values('count', ascending=False)
print(technique_summary)

## Step 7: Severity-Based Analysis

One of the key benefits is automatic severity assignment. Let's see how events are prioritized by severity level.

In [ ]:
# Severity distribution
print("SEVERITY DISTRIBUTION")
print("="*80)
severity_counts = mapped_events['severity'].value_counts()
print(severity_counts)

# Show high and critical severity events first (most important)
print("\n" + "="*80)
print("HIGH & CRITICAL SEVERITY EVENTS (Priority Review)")
print("="*80)
high_priority = mapped_events[mapped_events['severity'].isin(['high', 'critical'])]
if len(high_priority) > 0:
    print(high_priority[['timestamp', 'host', 'event_id', 'technique_id', 
                         'technique_name', 'tactic', 'severity']].to_string(index=False))
else:
    print("No high or critical severity events found.")

## Step 8: Generate Comprehensive Report

Finally, let's generate a complete CSV report that can be used for further analysis, reporting, or integration with other security tools.

In [ ]:
# Generate the comprehensive CSV report
output_file = "mitre_enriched_report.csv"
print(f"Generating comprehensive report: {output_file}...")

severity_counts = write_csv_report(enriched_events, output_file)

print(f"\n[+] Report written to {output_file}")
print("\n[*] Detection Summary by Severity:")
for severity in ['critical', 'high', 'medium', 'low', 'informational']:
    count = severity_counts.get(severity, 0)
    if count > 0:
        print(f"    {severity.capitalize()}: {count}")

total = sum(severity_counts.values())
print(f"    Total detections: {total}")

# Display the report
report_df = pd.read_csv(output_file)
print("\n" + "="*80)
print("COMPLETE ENRICHED REPORT (First 10 rows)")
print("="*80)
report_df.head(10)

## Step 9: Deep Dive - Examining Specific Techniques

Let's examine specific MITRE ATT&CK techniques in detail to understand what they mean and why they're important.

In [ ]:
# Get detailed information about a specific technique
def examine_technique(technique_id):
    """Get detailed information about a MITRE ATT&CK technique."""
    # Find events with this technique
    tech_events = mapped_events[mapped_events['technique_id'] == technique_id]
    
    if len(tech_events) == 0:
        print(f"No events found for technique {technique_id}")
        return
    
    print(f"\n{'='*80}")
    print(f"TECHNIQUE: {technique_id}")
    print(f"{'='*80}")
    
    first_event = tech_events.iloc[0]
    print(f"Technique Name: {first_event['technique_name']}")
    print(f"Tactic(s): {first_event['tactic']}")
    print(f"Severity: {first_event['severity']}")
    print(f"\nDescription:\n{first_event['description'][:500]}..." if len(str(first_event['description'])) > 500 else f"\nDescription:\n{first_event['description']}")
    
    print(f"\n\nEvents associated with this technique:")
    print(f"Total: {len(tech_events)}")
    print(tech_events[['timestamp', 'host', 'event_id', 'user', 'raw_message']].to_string(index=False))

# Examine some of the techniques we found
unique_techniques = mapped_events['technique_id'].unique()
print(f"\nFound {len(unique_techniques)} unique MITRE ATT&CK techniques")
print(f"Techniques: {', '.join(unique_techniques)}")

# Examine the first technique in detail
if len(unique_techniques) > 0:
    examine_technique(unique_techniques[0])

## Step 10: Understanding the Benefits - Before and After Comparison

Let's compare the original events with the enriched events to clearly see the value added by the MITRE mapper.

In [ ]:
print("BEFORE: Original Event (Limited Context)")
print("="*80)
print(f"Event ID: {original_events.iloc[0]['Event ID']}")
print(f"Description: {original_events.iloc[0]['Description']}")
print(f"\nWhat does this mean? An analyst would need to:")
print("  1. Look up Event ID 4624 in Microsoft documentation")
print("  2. Understand what this event indicates")
print("  3. Determine if it's related to known attack patterns")
print("  4. Manually assess severity and priority")

print("\n\nAFTER: Enriched Event (Actionable Threat Intelligence)")
print("="*80)
enriched_sample = mapped_events.iloc[0] if len(mapped_events) > 0 else enriched_df.iloc[0]
print(f"Event ID: {enriched_sample['event_id']}")
print(f"MITRE ATT&CK Technique: {enriched_sample['technique_id']} - {enriched_sample['technique_name']}")
print(f"Tactic: {enriched_sample['tactic']}")
print(f"Severity: {enriched_sample['severity']}")
print(f"\nDescription: {enriched_sample['description'][:300]}..." if len(str(enriched_sample['description'])) > 300 else f"\nDescription: {enriched_sample['description']}")

print("\n\nValue Added:")
print("  ✓ Immediate threat context (what attack technique this represents)")
print("  ✓ Automatic severity assignment (prioritization)")
print("  ✓ Standardized framework alignment (MITRE ATT&CK)")
print("  ✓ Detailed technique description (no manual lookup needed)")
print("  ✓ Tactic classification (attack phase identification)")

## Key Takeaways

### Why This Tool is Valuable for Security Operations

1. **Time Savings**: Automates the manual process of looking up Event IDs and understanding their security implications

2. **Consistency**: Provides standardized threat intelligence context using the industry-standard MITRE ATT&CK framework

3. **Prioritization**: Automatically assigns severity levels to help analysts focus on the most critical events first

4. **Scalability**: Can process thousands of events efficiently, making it practical for enterprise environments

5. **Threat Intelligence Integration**: Connects raw Windows events to known attack patterns, enabling proactive threat hunting

6. **Detection Engineering**: Helps security teams understand which events map to which attack techniques, improving detection coverage

7. **Reporting**: Generates structured reports that can be integrated into SIEM systems, ticketing systems, or security dashboards

8. **Continuous Updates**: Dynamic rule engine stays current with emerging threats through the Sigma repository

### Use Cases

- **Security Incident Response**: Quickly understand what attack techniques are present in event logs
- **Threat Hunting**: Identify patterns of MITRE ATT&CK techniques across large datasets
- **Detection Engineering**: Map existing events to ATT&CK techniques to identify detection gaps
- **Security Reporting**: Generate standardized threat intelligence reports for stakeholders
- **SIEM Enrichment**: Enrich events in SIEM systems with MITRE ATT&CK context
- **Forensic Analysis**: Understand the attack chain by mapping events to ATT&CK tactics

### Next Steps

1. Process your own Windows Security event logs
2. Integrate the script into automated security workflows
3. Customize severity mappings based on your organization's risk profile
4. Use the enriched reports for threat hunting and detection engineering
5. Schedule regular processing to maintain continuous security monitoring

---

**Note**: This tool is designed for security automation and threat intelligence. Always validate detections and consider false positives in your specific environment.